In [2]:
!pip install pyodbc # ODBC的DRIVER(驅動程式)

Active code page: 936


ERROR: Invalid requirement: '#'


### ----開始執行----

In [3]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
%autosave 120

【現在時間】2022/12/17 22:32:18
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


In [4]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

### ----更改股票代號----

In [23]:
stock_id = '1218' #1605 1218 6446
url = f"https://goodinfo.tw/tw/ShowBuySaleChart.asp?STOCK_ID={stock_id}&CHT_CAT=DATE" 
payloads = {"STOCK_ID": stock_id}
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
resp = requests.request(method="POST", url=url, data=payloads, headers=headers)
resp.encoding = "utf-8"
test = pd.read_html(resp.text, header=1)

In [24]:
df = test[5].dropna(axis='columns')
df.head(2)

# 三大法人買賣張數
"    0    1   2    3     4      5     6    7       8     9     10        11    12    13      14    15      16      17     18"

'    0    1   2    3     4      5     6    7       8     9     10        11    12    13      14    15      16      17     18'

In [25]:
products_list = df.values.tolist()
print(products_list[59])
del products_list[59]
del products_list[58]
del products_list[39]
del products_list[38]
del products_list[19]
del products_list[18]
lists_num=len(products_list)
print(len(products_list))
print(products_list)

['期別', '成交', '漲跌', '漲跌(%)', '成交量(張)', '買進(張)', '賣出(張)', '買賣超(張)', '持有(張)', '持股比率', '買進(張)', '賣出(張)', '買賣超(張)', '買進(張)', '賣出(張)', '買賣超(張)', '買進(張)', '賣出(張)', '買賣超(張)']
64
[['12/16', '36.5', '-1.7', '-4.45', '4014', '465', '1348', '-883', '13168', '2.63', '0', '0', '0', '4', '116', '-112', '469', '1463', '-994'], ['12/15', '38.2', '-0.7', '-1.8', '4072', '701', '1098', '-397', '14132', '2.82', '0', '0', '0', '4', '6', '-2', '705', '1104', '-399'], ['12/14', '38.9', '+0.35', '+0.91', '4127', '1390', '512', '+878', '14583', '2.91', '0', '26', '-26', '21', '7', '+14', '1411', '545', '+866'], ['12/13', '38.55', '-0.45', '-1.15', '3351', '508', '1031', '-523', '13705', '2.74', '0', '27', '-27', '3', '0', '+3', '511', '1058', '-547'], ['12/12', '39', '-0.25', '-0.64', '6364', '1297', '1589', '-292', '14228', '2.84', '0', '0', '0', '1', '0', '+1', '1298', '1589', '-291'], ['12/09', '39.25', '-0.9', '-2.24', '12968', '2547', '2911', '-364', '14534', '2.9', '0', '0', '0', '0', '19', '-19', '2547'

In [11]:
connsqlodbc= pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")


In [12]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令


sqlStr ='''create table 三大法人買賣張數表
(
	三大法人買賣張數表編號 nvarchar(20) primary key
	,[買進(張)] nvarchar(20) NULL
	,[賣出(張)] nvarchar(20) Null
	,[買賣超(張)] nvarchar(20) Null
    ,[買進(張).1] nvarchar(20) NULL
	,[賣出(張).1] nvarchar(20) Null
	,[買賣超(張).1] nvarchar(20) Null
    ,[買進(張).2] nvarchar(20) NULL
	,[賣出(張).2] nvarchar(20) Null
	,[買賣超(張).2] nvarchar(20) Null
    ,[買進(張).3] nvarchar(20) NULL
	,[賣出(張).3] nvarchar(20) Null
	,[買賣超(張).3] nvarchar(20) Null
	,三大法人日期 nvarchar(5) not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

In [26]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# INV+代號+發周別
# MYSQL指令
for i in range(0,lists_num):
    INVstock_id='INV'+str(stock_id)+str(products_list[i][0])
    sqlStr = f""" 
    Insert into 三大法人買賣張數表 (三大法人買賣張數表編號, 
    [買進(張)], [賣出(張)],
    [買賣超(張)],[買進(張).1],
    [賣出(張).1],[買賣超(張).1],
    [買進(張).2],[賣出(張).2],
    [買賣超(張).2],[買進(張).3],[賣出(張).3],
    [買賣超(張).3],三大法人日期,股票代號)
    values ('{str(INVstock_id)}',
    '{products_list[i][5]}','{products_list[i][6]}','{products_list[i][7]}','{products_list[i][10]}',
    '{products_list[i][11]}','{products_list[i][12]}','{products_list[i][13]}','{products_list[i][14]}',
    '{products_list[i][15]}','{products_list[i][16]}','{products_list[i][17]}','{products_list[i][18]}',
    '{products_list[i][0]}','{stock_id}')
    
    """
    cursor.execute(sqlStr)

print(sqlStr)

 
    Insert into 三大法人買賣張數表 (三大法人買賣張數表編號, 
    [買進(張)], [賣出(張)],
    [買賣超(張)],[買進(張).1],
    [賣出(張).1],[買賣超(張).1],
    [買進(張).2],[賣出(張).2],
    [買賣超(張).2],[買進(張).3],[賣出(張).3],
    [買賣超(張).3],三大法人日期,股票代號)
    values ('INV121809/19',
    '159','203','-44','2',
    '0','+2','0','1.06',
    '-1.06','161','204','-43.1',
    '09/19','1218')
    
    


In [ ]:
connsqlodbc.close()

In [ ]:
'''
期別
買進(張)賣出(張)買賣超(張)
買進(張).1賣出(張).1買賣超(張).1
買進(張).2賣出(張).2買賣超(張).2
買進(張).3賣出(張).3買賣超(張).3
'''